In [244]:
import numpy as np
class Node:
    def __init__(self, d = None):
        self.left = None
        self.right = None
        self.data = d
        self.parent = None

    def getRightmostChild(self):
        if self.right == None and self.left == None:
            return self
        if self.right:
            return self.right.getRightmostChild()
        return self.left.getRightmostChild()

    def getLeftmostChild(self):
        if self.right == None and self.left == None:
            return self
        if self.left:
            return self.left.getLeftmostChild()
        return self.right.getLeftmostChild()

    
    def getNearestLeftNode(self):
        foundParentWithLeftChildren = False
        curr = self
        while not foundParentWithLeftChildren:
            if curr.parent == None:
                return None
            if curr.parent.left and curr.parent.left != curr:
                curr = curr.parent.left
                foundParentWithLeftChildren = True
                break
            curr = curr.parent
        return curr.getRightmostChild()

    def getNearestRightNode(self):
        foundParentWithRightChildren = False
        curr = self
        while not foundParentWithRightChildren:
            if curr.parent == None:
                return None
            if curr.parent.right and curr.parent.right != curr:
                curr = curr.parent.right
                foundParentWithRightChildren = True
                break
            curr = curr.parent
        return curr.getLeftmostChild()
    
    def add(self, rightNode):
        newNode = Node()
        newNode.left = self
        newNode.right = rightNode
        self.parent = newNode
        rightNode.parent = newNode
        return newNode
    
    def isLeftChild(self):
        if self.parent.left == self:
            return True
        return False
    
    
    def explosionFree(self, level=0, root=None):
        if self.data == None and level >= 4:
            if DEBUG:
                print("Exploding", self)
            if self.left.data == None or self.right.data == None:
                print("Error, spliting a pair that doesn't have numbers as children is unexpected")
                exit(1)
            leftNN = self.getNearestLeftNode()
            rightNN = self.getNearestRightNode()
            if leftNN:
                leftNN.data += self.left.data
            if rightNN:
                rightNN.data += self.right.data

            newNode = Node(0)
            newNode.parent = self.parent
            if self.isLeftChild():
                self.parent.left = newNode
            else: 
                self.parent.right = newNode
            if DEBUG:
                print("After explosion", root)
            return False
        
        if self.left:
            if not self.left.explosionFree(level+1, root):
                return False
        if self.right:
            if not self.right.explosionFree(level+1, root):
                return False
        return True

    def splitFree(self, root=None):
        #Check split on data
        if self.data != None and self.data >= 10:
            #Make not necessarily reducedtree
            #newNode = makeTreeFromString("[" + str() + "," + str(int(np.ceil(self.data/2.0))) + "]")
            leftChild = Node(int(np.floor(self.data/2.0)))
            rightChild = Node(int(np.ceil(self.data/2.0)))
            newNode = Node()
            newNode.left, newNode.right = leftChild, rightChild
            leftChild.parent = newNode
            rightChild.parent = newNode
            
            newNode.parent = self.parent
            if self.isLeftChild():
                self.parent.left = newNode
            else:
                self.parent.right = newNode
            if DEBUG:
                print("After split", root)
            return False
        if self.left:
            if not self.left.splitFree(root):
                return False
        if self.right:
            if not self.right.splitFree(root):
                return False
        return True
    
    
    #Returns true if tree is reduced
    def reduce(self, root=None): 
        if not self.explosionFree(0,root):
            return False
        if not self.splitFree(root):
            return False
        return True
            
    def fullReduce(self, root=None):
        reduceFinished = False
        cnt = 0
        while not reduceFinished:
            reduceFinished = self.reduce(root)
            cnt += 1
            if cnt %100 == 0:
                print("Performed {} reductions".format(cnt))
        return self
            

    def __str__(self):
        return self.inorderStr()
    
        
    def inorderStr(self):
        val = ""
        if self:
            if self.data != None:
                val += self.data.__str__()
            else:        
                val += "["
                if self.left:
                    val += self.left.inorderStr()
                val += ","
                if self.right:
                    val += self.right.inorderStr()
                val += "]"
        return val

def makeTreeFromString(line):
    root = None
    prev = None
    for char in line:
        if char.isspace() or char == ",":
            continue
        if char == "[":
            curr = Node()
            if prev == None:
                root = curr
            else:
                curr.parent = prev
                if prev.left:
                    prev.right = curr
                else:
                    prev.left = curr
            prev = curr

        elif char ==  "]":
            if prev:
                prev = prev.parent

        else:
            newNode =Node(int(char))
            if prev.left:
                prev.right = newNode
            else:
                prev.left = newNode
            newNode.parent = prev
    return root    
    
def readInput(input):
    nodes = []
    lines = input.split("\n")
    for line in lines:
        nodes.append(makeTreeFromString(line))
    return nodes

def addNodes(nodes):
    A = nodes[0]
    for n in range(1,len(nodes)):
        A=A.add(nodes[n])
        print("ADDED TOGETHER GIVES", A)
        A=A.fullReduce()
        print(A)
    return A

def getMagnitude(node):
    if node.data != None:
        return node.data
    return 3*getMagnitude(node.left) + 2*getMagnitude(node.right)



DEBUG=False
#nodes = readInput(input)
#[n.__str__() for n in nodes]
#root = makeTreeFromString("[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]")
#root = makeTreeFromString("[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]")
#root = makeTreeFromString("[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]")
#root = makeTreeFromString(" [[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]],[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]]")


nodes = readInput(input)
root = addNodes(nodes)
#print(root)
root = root.fullReduce(root)
print(getMagnitude(root))

ADDED TOGETHER GIVES [[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]],[[[5,[2,8]],4],[5,[[9,9],0]]]]
[[[[7,0],[7,8]],[[7,9],[0,6]]],[[[7,0],[6,6]],[[7,7],[0,9]]]]
ADDED TOGETHER GIVES [[[[[7,0],[7,8]],[[7,9],[0,6]]],[[[7,0],[6,6]],[[7,7],[0,9]]]],[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]]
Performed 100 reductions
[[[[7,7],[7,7]],[[7,0],[7,7]]],[[[7,7],[6,7]],[[7,7],[8,9]]]]
ADDED TOGETHER GIVES [[[[[7,7],[7,7]],[[7,0],[7,7]]],[[[7,7],[6,7]],[[7,7],[8,9]]]],[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]]
Performed 100 reductions
Performed 200 reductions
[[[[6,6],[6,6]],[[7,7],[7,7]]],[[[7,0],[7,7]],[[7,8],[8,8]]]]
ADDED TOGETHER GIVES [[[[[6,6],[6,6]],[[7,7],[7,7]]],[[[7,0],[7,7]],[[7,8],[8,8]]]],[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]]
Performed 100 reductions
Performed 200 reductions
[[[[6,6],[7,7]],[[7,7],[8,8]]],[[[8,8],[0,8]],[[8,9],[9,9]]]]
ADDED TOGETHER GIVES [[[[[6,6],[7,7]],[[7,7],[8,8]]],[[[8,8],[0,8]],[[8,9],[9,9]]]],[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]]
Performed 100 reductions
Performed 200 r

In [245]:
#A= makeTreeFromString("[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]")
#B= makeTreeFromString("[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]")
#root = A.add(B)
#root = root.fullReduce()
#print(getMagnitude(root))


import copy
maxSum = 0
nodes = readInput(input)
c1,c2 = 0,0
for i in range(0,len(nodes)):
    for j in range(i+1, len(nodes)):
        A, B = copy.deepcopy(nodes[i]), copy.deepcopy(nodes[j])
        val1 = getMagnitude(A.add(B).fullReduce()) 
        val2 = getMagnitude(B.add(A).fullReduce()) 
        if  val1 > maxSum:
            maxSum = val1
            c1,c2 = i,j
        if  val2 > maxSum:
            maxSum = val2
            c1,c2 = j,i
print(maxSum)



Performed 100 reductions
Performed 100 reductions
3891


In [89]:
input = """[1,2]
[[1,2],3]
[9,[8,7]]
[[1,9],[8,5]]
[[[[1,2],[3,4]],[[5,6],[7,8]]],9]
[[[9,[3,8]],[[0,9],6]],[[[3,7],[4,9]],3]]
[[[[1,3],[5,3]],[[1,3],[8,7]]],[[[4,9],[6,9]],[[8,2],[7,3]]]]"""

In [187]:
input = """[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
[7,[5,[[3,8],[1,4]]]]
[[2,[2,2]],[8,[8,1]]]
[2,9]
[1,[[[9,3],9],[[9,0],[0,7]]]]
[[[5,[7,4]],7],1]
[[[[4,2],2],6],[8,7]]"""

In [215]:
input = """[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]"""

In [243]:
input = """[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]"""